In [1]:
import pandas as pd
import uuid
#from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import tensorflow_decision_forests as tfdf

# Used in FL
import nest_asyncio
nest_asyncio.apply()
import collections
import tensorflow_federated as tff
import tensorflow as tf

np.random.seed(0)
import random

2023-04-15 21:45:43.625279: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-15 21:45:43.828574: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-15 21:45:43.829899: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-15 21:45:49.522204: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Testing TFF
# tff.backends.native.set_local_python_execution_context()
print(tff.federated_computation(lambda: 'Hello, World!')())

2023-04-15 21:46:05.370366: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2023-04-15 21:46:05.372068: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
E0415 21:46:05.622696800     143 socket_utils_common_posix.cc:221] check for SO_REUSEPORT: {"created":"@1681609565.622667100","description":"Protocol not available","errno":92,"file":"external/com_github_grpc_grpc/src/core/lib/iomgr/socket_utils_common_posix.cc","file_line":199,"os_error":"Protocol not available","syscall":"getsockopt(SO_REUSEPORT)"}
E0415 21:46:05.623348200     143 socket_utils_common_posix.cc:327] setsockopt(TCP_USER_TIMEOUT) Protocol not available


b'Hello, World!'


In [3]:
# Read the CSV files into separate dataframes
df_ey = pd.read_csv('/home/draxtik20/Federated_Learning_CDI/Datasets/EIRLI_younger.csv')
df_eo = pd.read_csv('/home/draxtik20/Federated_Learning_CDI/Datasets/EIRLI_older.csv')
df_ly = pd.read_csv('/home/draxtik20/Federated_Learning_CDI/Datasets/LASER_younger.csv')
df_lo = pd.read_csv('/home/draxtik20/Federated_Learning_CDI/Datasets/LASER_older.csv')

In [4]:
# Concatenate the dataframes into one dataframe with matching headers
df = pd.concat([df_ey, df_eo, df_ly, df_lo], ignore_index=True, sort=False)

In [5]:
df

,Level.gender,Level.ethnic,Level.med,Level.fed,Level.income,FamHist,Percentile,GCC,Degree,HC,Between,PathLength,CombWords,AnyLangorReadDxOnly,GramComplex
0,0,4,4,4,3,0,67.0,0.431818,3.632653,0.258744,27.061224,2.870240,2.0,NoDx,NaN
1,1,4,6,6,3,0,95.0,0.597950,12.480000,0.336051,144.480000,2.987580,1.0,NoDx,NaN
2,1,4,5,4,2,1,10.0,0.000000,1.000000,0.416667,0.250000,1.333333,1.0,NoDx,NaN
3,0,4,3,3,2,0,37.0,0.473684,2.000000,0.212454,1.142857,1.615385,1.0,NoDx,NaN
4,1,4,6,3,3,0,92.0,0.543568,8.125000,0.293187,128.159722,3.270546,2.0,NoDx,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
869,0,4,6,4,3,1,1.0,0.691011,2.666667,0.140688,5.416667,2.266234,1.0,NoDx,15.0
870,0,4,4,6,3,0,95.0,0.516904,17.176471,0.319899,300.974790,3.040681,1.0,NoDx,55.0
871,1,4,5,3,2,0,1.0,0.818182,1.933333,0.106475,5.166667,2.823529,1.0,Dx,39.0
872,1,1,3,2,1,1,48.0,0.565451,13.000000,0.310525,191.393162,3.060358,1.0,NoDx,55.0


In [6]:
# Generate a unique client ID for each row
df['client_id'] = [str(uuid.uuid4()) for _ in range(len(df))]

# Set the index of the dataframe to the client_id column
df.set_index('client_id', inplace=True)

In [7]:
# Print the dataframe
df

,Level.gender,Level.ethnic,Level.med,Level.fed,Level.income,FamHist,Percentile,GCC,Degree,HC,Between,PathLength,CombWords,AnyLangorReadDxOnly,GramComplex
client_id,,,,,,,,,,,,,,,
3532add5-1f47-43ea-a997-6d611d024135,0,4,4,4,3,0,67.0,0.431818,3.632653,0.258744,27.061224,2.870240,2.0,NoDx,NaN
9b6b2b10-f5dc-4053-97f1-6bc5ac9828b7,1,4,6,6,3,0,95.0,0.597950,12.480000,0.336051,144.480000,2.987580,1.0,NoDx,NaN
9c657661-bb91-44dc-90d1-fc22f3dad18e,1,4,5,4,2,1,10.0,0.000000,1.000000,0.416667,0.250000,1.333333,1.0,NoDx,NaN
43e7d58b-4a8d-4dec-9379-b1e85b137362,0,4,3,3,2,0,37.0,0.473684,2.000000,0.212454,1.142857,1.615385,1.0,NoDx,NaN
4a763889-653a-4155-be97-dd4770681ce9,1,4,6,3,3,0,92.0,0.543568,8.125000,0.293187,128.159722,3.270546,2.0,NoDx,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
03836774-fb74-493e-af07-da2556e5615d,0,4,6,4,3,1,1.0,0.691011,2.666667,0.140688,5.416667,2.266234,1.0,NoDx,15.0
2f2e51cf-b600-4c31-9456-5427d3560e64,0,4,4,6,3,0,95.0,0.516904,17.176471,0.319899,300.974790,3.040681,1.0,NoDx,55.0
67a10a1a-62bb-4509-93a8-297d6b6d7ee8,1,4,5,3,2,0,1.0,0.818182,1.933333,0.106475,5.166667,2.823529,1.0,Dx,39.0


In [8]:
# Get the columns with NaN values
nan_cols = df.columns[df.isnull().any()]

# Print the columns with NaN values as a list
print(nan_cols.tolist())

['GramComplex']


In [9]:
numeric_cols = df.select_dtypes(include=np.number).columns.tolist()
# Fill missing values in the numeric columns with the mean value
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].mean())

In [10]:
# Replace the 'NoDx' value with 0 and the 'Dx' value with 1
df["AnyLangorReadDxOnly"] = df["AnyLangorReadDxOnly"].replace("NoDx", 0).replace("Dx", 1)
df["AnyLangorReadDxOnly"] = df["AnyLangorReadDxOnly"].astype(int)

In [11]:
# Split the data into training and testing sets
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

In [12]:
print(len(train_data), len(test_data), type(train_data))

699 175 <class 'pandas.core.frame.DataFrame'>


In [13]:
print(test_data.dtypes)

Level.gender             int64
Level.ethnic             int64
Level.med                int64
Level.fed                int64
Level.income             int64
FamHist                  int64
Percentile             float64
GCC                    float64
Degree                 float64
HC                     float64
Between                float64
PathLength             float64
CombWords              float64
AnyLangorReadDxOnly      int64
GramComplex            float64
dtype: object


In [14]:
# Convert the training data to a TensorFlow Dataset
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_data, label="AnyLangorReadDxOnly")

print(train_ds)

<_PrefetchDataset element_spec=({'Level.gender': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'Level.ethnic': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'Level.med': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'Level.fed': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'Level.income': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'FamHist': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'Percentile': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'GCC': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'Degree': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'HC': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'Between': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'PathLength': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'CombWords': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'GramComplex': TensorSpec(shape=(None,), dtype=tf.float64, name=None)}, TensorSpec(shape=(N

In [15]:
# ----------------------------------------------------Federated Data-----------------------------------------------------------

In [16]:
#create a list of client_names
client_names = df.index.values

# Get a numpy array of each client's values for all collumns, except the label column
client_data = df[["Level.gender", "Level.ethnic", "Level.med", "Level.fed", "Level.income", "FamHist",\
                 "Percentile", "GCC", "Degree", "HC", "Between", "PathLength", "CombWords", "GramComplex"]].values

# Get a numpy array of each client's value for only the label column
client_labels = df["AnyLangorReadDxOnly"].values

# Get an array of a tuple(client_data element, respective client_label)
data = list(zip(client_data, client_labels))

#shard data and place at each client
shards = [data[i:i + 1] for i in range(0, len(data), 1)]

# Assert number of clients must equal number of shards
assert(len(shards) == len(client_names))

# Create a dictionary with the client names as keys and the data columns as the values
clients = {client_names[i] : shards[i] for i in range(len(client_names))}

In [17]:
print(clients)

{'3532add5-1f47-43ea-a997-6d611d024135': [(array([ 0.        ,  4.        ,  4.        ,  4.        ,  3.        ,
        0.        , 67.        ,  0.43181818,  3.63265306,  0.25874433,
       27.06122449,  2.87023977,  2.        , 58.05600077]), 0)], '9b6b2b10-f5dc-4053-97f1-6bc5ac9828b7': [(array([  1.        ,   4.        ,   6.        ,   6.        ,
         3.        ,   0.        ,  95.        ,   0.59795018,
        12.48      ,   0.33605145, 144.48      ,   2.98757959,
         1.        ,  58.05600077]), 0)], '9c657661-bb91-44dc-90d1-fc22f3dad18e': [(array([ 1.        ,  4.        ,  5.        ,  4.        ,  2.        ,
        1.        , 10.        ,  0.        ,  1.        ,  0.41666667,
        0.25      ,  1.33333333,  1.        , 58.05600077]), 0)], '43e7d58b-4a8d-4dec-9379-b1e85b137362': [(array([ 0.        ,  4.        ,  3.        ,  3.        ,  2.        ,
        0.        , 37.        ,  0.47368421,  2.        ,  0.21245421,
        1.14285714,  1.61538461,  1.

In [29]:
# Batch the data_shard into a an array of tensorflow dataset
def batch_data(data_shard, bs=32, pref=25):
    '''Takes in a clients data shard and create a tfds object off it
    args:
        shard: a data, label constituting a client's data shard
        bs:batch size
    return:
        tfds object'''
    def batch_format_fn(element):
        print(element['AnyLangorReadDxOnly'])
        return collections.OrderedDict(
            x=tf.reshape(element[["Level.gender", "Level.ethnic", "Level.med", "Level.fed", "Level.income", "FamHist",\
                     "Percentile", "GCC", "Degree", "HC", "Between", "PathLength", "CombWords", "GramComplex"]], [-1, 14]),
            y=tf.reshape(element['AnyLangorReadDxOnly'], (-1, 1)))#element['label'])
    #seperate shard into data and labels lists
    data, label = zip(*data_shard)
    dataset = tf.data.Dataset.from_tensor_slices((list(data), list(label)))
    # Shuffle the dataset, batch the 
    dataset = dataset.repeat(5).shuffle(len(label)).batch(bs).prefetch(pref)
    
#     dataset = dataset.map(lambda x, y: collections.OrderedDict([('x',x),('y',y)]))
    
#     dataset = dataset.map(batch_format_fn)
    return dataset

# Run the function for each row in the clients dictionary, and add it back to a dictionary clients_batched
federated_data = []
for (client_name, data) in clients.items():
    federated_data.append(batch_data(data))
    
print(federated_data[0])

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 14), dtype=tf.float64, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>


In [31]:
print(federated_data[0])

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 14), dtype=tf.float64, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>


In [20]:
# -----------------------------------------------Create a Federated Model-------------------------------------------------------

In [21]:
federated_data

[<_MapDataset element_spec=OrderedDict([('x', TensorSpec(shape=(None, 14), dtype=tf.float64, name=None)), ('y', TensorSpec(shape=(None,), dtype=tf.int32, name=None))])>,
 <_MapDataset element_spec=OrderedDict([('x', TensorSpec(shape=(None, 14), dtype=tf.float64, name=None)), ('y', TensorSpec(shape=(None,), dtype=tf.int32, name=None))])>,
 <_MapDataset element_spec=OrderedDict([('x', TensorSpec(shape=(None, 14), dtype=tf.float64, name=None)), ('y', TensorSpec(shape=(None,), dtype=tf.int32, name=None))])>,
 <_MapDataset element_spec=OrderedDict([('x', TensorSpec(shape=(None, 14), dtype=tf.float64, name=None)), ('y', TensorSpec(shape=(None,), dtype=tf.int32, name=None))])>,
 <_MapDataset element_spec=OrderedDict([('x', TensorSpec(shape=(None, 14), dtype=tf.float64, name=None)), ('y', TensorSpec(shape=(None,), dtype=tf.int32, name=None))])>,
 <_MapDataset element_spec=OrderedDict([('x', TensorSpec(shape=(None, 14), dtype=tf.float64, name=None)), ('y', TensorSpec(shape=(None,), dtype=tf.int

In [ ]:
#--------------------------------------------Rest of Centralized Model----------------------------------------------------------

In [ ]:
model = tfdf.keras.RandomForestModel(num_trees=50, max_depth=10)

In [ ]:
# Fit the model to the training data
model.fit(train_ds)

In [ ]:
# Convert the testing data to a TensorFlow Dataset
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test_data)

# Make predictions on the testing data
y_pred = model.predict(test_ds)

In [ ]:
tfdf.model_plotter.plot_model_in_colab(model, tree_idx=0, max_depth=2)

In [ ]:
model.summary()

In [ ]:
model.compile(metrics=["accuracy"])
evaluation = model.evaluate(test_data, return_dict=True)
print()

for name, value in evaluation.items():
  print(f"{name}: {value:.4f}")